In [1]:
family_group = "8.Heliconiinae_Danainae"


def get_genome_file(genome_location,species):
    list_of_files_in_genome_folder = os.listdir(f"{genome_location}/{species}")
    for file in list_of_files_in_genome_folder:
        if file.endswith("_genomic.fna"):
            genome_file = file
    return(genome_file)  

def get_gene_sequence(genome_location, 
                      species, 
                      genome_file, 
                      scaffold,
                      gene_start,
                      gene_end,
                      complement,
                      output_location):

    
    print("Getting Gene")
    list_of_folders = os.listdir(output_location)
    # print(list_of_files_inside_annotated_species_folder)
    # print(
    if f"0.Temp" not in list_of_folders:
        os.mkdir(f"{output_location}/0.Temp")

#     subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}"', shell = True, stderr = subprocess.DEVNULL)
#     print(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end}')
    subprocess.run(f'samtools faidx "{genome_location}/{species}/{genome_file}" {scaffold}:{gene_start}-{gene_end} > "{output_location}/0.Temp/temp_genome.fa"', shell = True, stderr = subprocess.DEVNULL)
    
    genome = SeqIO.parse(f"{output_location}/0.Temp/temp_genome.fa", "fasta")
    for entries in genome:
        gene_sequence = entries.seq
        if complement == "1":
            gene_sequence = gene_sequence.reverse_complement()
        break
    # print(gene_sequence)
    return (gene_sequence)
def make_output_folder(output_location):
    list_of_folders = os.listdir(output_location)
    if "2.Final_output" in list_of_folders:
        subprocess.run(f'rm -r "{output_location}/2.Final_output"', shell = True, stderr = subprocess.DEVNULL)
    os.mkdir(f"{output_location}/2.Final_output")

def write_exon_output(output_location,species,exon,exon_output):
    
    
    list_of_folders_in_final_output = os.listdir(f"{output_location}/2.Final_output")
    if species not in list_of_folders_in_final_output:
        os.mkdir(f"{output_location}/2.Final_output/{species}")
    
    with open(f"{output_location}/2.Final_output/{species}/{exon}.fa", 'w') as out_file:
        out_file.write(exon_output)

In [2]:
import os
from Bio import SeqIO
import subprocess



output_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/{family_group}"
make_output_folder(output_location)
genome_location = "/mnt/f/Genomes_2023-12-26"
list_of_species = os.listdir(f"{output_location}/1.Blast_result")
if 'desktop.ini' in list_of_species:
    list_of_species.remove("desktop.ini")
# print(list_of_species)
# list_of_species = ["Heliconius_charithonia"]
# list_of_species = ["Vanessa_tameamea"]
error_exons = ''
for species in list_of_species:
    print(species)
    genome_file = get_genome_file(genome_location,species)
    print(genome_file)
    previous_coordinate_start = ''
    previous_scaffold = ''
    with open(f"{output_location}/1.Blast_result/{species}/final_coordinates.csv", 'r') as coordinate_final_file:
        coordinate_file_list = coordinate_final_file.readlines()
    
    for i in range(1,len(coordinate_file_list)):
        
        coordinate_line_split = coordinate_file_list[i].split(",")
        
            
#         print(coodinate_line_split[5])
        if coordinate_line_split[5] == "N":
            if previous_coordinate_start == '':
                previous_coordinate_start = coordinate_line_split[2]
                previous_scaffold = coordinate_line_split[1]
            complement = coordinate_line_split[4]
            start,stop = coordinate_line_split[2],coordinate_line_split[3]
            print(start,stop)
            query_name = coordinate_line_split[6]
            scaffold = coordinate_line_split[1]
            exon = f"Exon_{query_name.split('_')[-1]}"
            left_oh, right_oh = query_name.split('_')[-6], query_name.split('_')[-4]
#             print(complement)
            if complement == "0":
                if start < previous_coordinate_start or scaffold != previous_scaffold:
                    print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                    prompt = input("Continue (Y/N)?")
                    if prompt[0].lower() == "n":
                        assert False
            elif complement == "1":
                if start > previous_coordinate_start or scaffold != previous_scaffold:
                    print(f"Error in Coordinates\nPrevious_coordinate = {coordinate_file_list[i-1]}\nCurrent_coordinate = {coordinate_file_list[i]}")
                    prompt = input("Continue (Y/N)?")
                    if prompt[0].lower() == "n":
                        assert False
            else:
                print("ERROR")
            
        elif coordinate_line_split[5] == "Y":
            error_exons += coordinate_file_list[i]
        
        gene_sequence = get_gene_sequence(genome_location, 
                                          species, 
                                          genome_file, 
                                          scaffold,
                                          start,
                                          stop,
                                          complement,
                                          output_location)
        
       
        current_exon = query_name.split('_')[-1]
        if current_exon == "1":
            if complement == 1:
                translated = gene_sequence.reverse_complement.translate()
            else:
                translated = gene_sequence.translate()
            print(translated)
            if "*" in translated:
                error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
            else:
                exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                print(exon_output)
        
                write_exon_output(output_location,species,exon,exon_output)
        elif current_exon == "24":
            if complement == 1:
                translated = gene_sequence.reverse_complement.translate()
            else:
                translated = gene_sequence.translate()

            if "*" in translated[:-1]:
                error_exons += f"{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n"
                print(translated)
            else:
                
                exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
                print(exon_output)
                write_exon_output(output_location,species,exon,exon_output)
#         assert False
        else:
            exon_output = f">{species}_{exon}_{scaffold}_{start}_{stop}_left_{left_oh}_right_{right_oh}\n{gene_sequence}"
            print(exon_output)

            write_exon_output(output_location,species,exon,exon_output)
        
#         assert False
        previous_coordinate_start = start
        previous_scaffold = scaffold
    error_exons += "\n\n"
#     print(coordinate_file_list)
#     break
coordinate_file = ''
print(error_exons)
with open(f"{output_location}/error_exons.txt", 'w') as error_out_file:
    error_out_file.write(error_exons)

Danaus_chrysippus
GCA_916720795.1_Dchry2.2_genomic.fna
6369470 6369558
Getting Gene


/home/saurav/.local/lib/python3.8/site-packages/Bio/Seq.py:2880: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


MDNLDDSENNAKISDSAYSNSCSNSQSRR
>Danaus_chrysippus_Exon_1_CAKASE010000043.1_6369470_6369558_left_0_right_2
ATGGACAACTTGGATGACTCCGAGAACAATGCTAAGATATCCGATTCCGCATATTCCAACAGCTGCAGTAACAGTCAGTCGAGGAGAAG
6369627 6369710
Getting Gene
>Danaus_chrysippus_Exon_2_CAKASE010000043.1_6369627_6369710_left_1_right_2
cCACAGCTCCAAATCTACACATTCTGGAAGCAATTCAAGTGGAAGCAGTGGTTATGGCGGCAAGCCATCGACTTCCGGCTACAG
6369781 6369941
Getting Gene
>Danaus_chrysippus_Exon_3_CAKASE010000043.1_6369781_6369941_left_1_right_1
CAATAACTTAAGTCAGCCACCGGAAAAATGGATTAAAGAAAAGGAATCAAAAAAGAAAAAGCCTGTACAGGTAGAATTGAAACCATCTGAAGAGAAAATAGAAGAATGTCCACAAGAACTCACACCTGTGTGTGAGGCGCCTAAAGAAGAAGCTAAAGATG
6370670 6370764
Getting Gene
>Danaus_chrysippus_Exon_4_CAKASE010000043.1_6370670_6370764_left_2_right_0
TTAACCGCACCCCGACTCCATCTTTGGTGCAAACCGAAAAGGGTCCGGAGAACATGGAAATAACAACGTTAAAAAGAACCGATGAAAAAGATGAG
6370892 6370966
Getting Gene
>Danaus_chrysippus_Exon_5_CAKASE010000043.1_6370892_6370966_left_0_right_0
TCTGTTTCGTCCAACGCGCCTATGGTGACATCGCTGAATTTAGTCA

6604437 6604586
Getting Gene
>Danaus_plexippus_Exon_12_NC_083559.1_6604437_6604586_left_0_right_0
GTCCTTACAAAACCAGCTGAAATAGCTAAACAGCAGATGAGTAAAAGATGCCAGGATCTCGCTTCCTTCATGGAGAGTCTGATGGAGGAGACACCCAAGATAGAAGAAGAACTACGTCTTGAAATACAGGATCAAGATCATAGTTATTAT
6605111 6605229
Getting Gene
>Danaus_plexippus_Exon_13_NC_083559.1_6605111_6605229_left_0_right_2
GAACGTGATTCTGTAATGCTCGGCGGTATCTCGCCACACCACGACCACAGCGACAGCAAGTCAGGCACCGACACACCAGTCAGCTACAACCAACTTAATTACAACGAAACTCTTCAAAG
6605353 6605451
Getting Gene
>Danaus_plexippus_Exon_14_NC_083559.1_6605353_6605451_left_1_right_2
ATATTTTGACAGTCATGAACCATACAGCTTcgaagattattatttaatggatAGCGAAAATAAGATCcaaatgaaagaaaatgaAGAAGGTTCTGTGAG
6605880 6606017
Getting Gene
>Danaus_plexippus_Exon_15_NC_083559.1_6605880_6606017_left_1_right_2
caAATGTATATCTCCGATGGCACAAGCTTCGACGGAGTGTGATCGGACCAGTTCCTCCGAATGCAGTGGTCTTGGTATAGGGAACTCTTGTCCTTGTGACTACCAGCCAATGCGATTGACAGAGTCATTGCTTAACAA
6606331 6606499
Getting Gene
>Danaus_plexippus_Exon_16_NC_083559.1_6606331_6606499_left_1_right_0


>Dircenna_loreta_Exon_22_OY743092.1_3846758_3846916_left_0_right_0
CACACAGAAGTGACAAAAGATATAACTAATAGGAAAATGCGACTAAGTAACAGCAGTAGAACCGTGGAAAAGACCGACGAGGAGTCTAGCTATTCGTCGTTTTATTCATCCTTTTTCAAAACTGAGTCCGGCAGCGCCGAAGAGAGCGATGCCAAGAAG
3847181 3847306
Getting Gene
>Dircenna_loreta_Exon_23_OY743092.1_3847181_3847306_left_0_right_0
agtTGGCCCAAAAATTCAAGAGCGGACGATGCGATGTCGCTGCAGAGTTCACTCGACGCACACACGTATCAACCGAAACATGCCAAGGCTAATAGAAAAAGGAATGATCCGCCCTGGATGGAAGAA
3848036 3848137
Getting Gene
>Dircenna_loreta_Exon_24_OY743092.1_3848036_3848137_left_0_right_0
gTTTGCGTTACTTCagaactaatatataaatatcagatCCGAACTAAAAATTTGGACGAAGTTCTGTCTgaagataaaaagaaactgGCAGCTCTGGAACaa
3848236 3848379
Getting Gene
>Dircenna_loreta_Exon_25_OY743092.1_3848236_3848379_left_0_right_0
CCCGCAATGGTTAACGAACAACTCGGACAACTCTACTTGGACTTGCAACTACAAGGAGTGGCCGCAAGATTGACTTTAGAGGAAGGTATCACAAGCTCGAGCAGTTCTGGTGAAGAAAGTACCTCGAGGTCATCTAAAGTGAAG
3849207 3849308
Getting Gene
>Dircenna_loreta_Exon_26_OY743092.1_3849207_3849308_left_0_right_0
ATTCGAAGAAGAAA

...

>Nymphalis_c-album_Exon_19_OY720586.1_12336257_12336385_left_2_right_1
GTCTAATGGGTCATCATGCTAGTAGGATGGGTTCTCATTACGAAGGTCATATGAACACTAAATGTTCCTCATCTTCAACTTCATACAGATCCAGTCCTTATGTGTCAACTCCGTATGAACCACGGCAAA
12335411 12335562
Getting Gene
>Nymphalis_c-album_Exon_20_OY720586.1_12335411_12335562_left_2_right_0
tgCCTTATATGATGTTTGGGCAGGCTGTCTATACATCGCCTTTCATGTATTCGTCCATCGATCCTCAAATATACACCCTGCAACAGAGCTTCGTCCCACAAAGTGTTCCGAACATGTCTTTATTggaaaattcaaatcaaaattacgAAGag
Getting Gene
>Nymphalis_c-album_Exon_20_OY720586.1_12335411_12335562_left_2_right_0
tgCCTTATATGATGTTTGGGCAGGCTGTCTATACATCGCCTTTCATGTATTCGTCCATCGATCCTCAAATATACACCCTGCAACAGAGCTTCGTCCCACAAAGTGTTCCGAACATGTCTTTATTggaaaattcaaatcaaaattacgAAGag
12332388 12332558
Getting Gene
>Nymphalis_c-album_Exon_22_OY720586.1_12332388_12332558_left_0_right_0
aaaaaaagatCAATAATGGCAAGTCGTAATAAGAGGAAGCTAAAAATGGGCAAGAGCGACGAATCGAAGACGGACGAGGAGTCGAGCTACTCTTCACTCTATTCATCATTTTTCAAGACTGATTCGGGTAGTAACGAGGAGAGCGACgggaaaaactttaaaaatgataaa
12331562 12331681
Getting Gene
>N

12186822 12186913
Getting Gene
>Nymphalis_polychloros_Exon_4_HG992241.1_12186822_12186913_left_2_right_0
CTATCCTGTCATCGCCGCCTCGTCTTGAACTAGAACAGAAAACGATTATTATGGACATTTCAACTTCGCTCCAGTGCAGCGTCAAGAAAGAG
12186300 12186365
Getting Gene
>Nymphalis_polychloros_Exon_5_HG992241.1_12186300_12186365_left_0_right_0
TGTGTCCAATCCAGCTCAACGCTTGTCTCTACACTGAACTGCGTCACTCGTACGGAACGTCGTTTG
12185823 12185954
Getting Gene
>Nymphalis_polychloros_Exon_6_HG992241.1_12185823_12185954_left_0_right_0
gaGGGATTTTCGTGTGTAATATCGATGCACGACGGTATCGTAATGTATACGACTACGTCTATAACGGCTGCGCTTGGTTTCCCGAAAGACATGTGGGTCGGCAGATCATTCATCGACTTCGTTCACCCAAGA
12185530 12185605
Getting Gene
>Nymphalis_polychloros_Exon_7_HG992241.1_12185530_12185605_left_0_right_1
GATCGCAATACATTCGCATCTCAGATAACAAATGGCTTGACAGTTCCAAAGAATGCTAACGGTATGCAAACAAAAG
12183305 12183517
Getting Gene
>Nymphalis_polychloros_Exon_8_HG992241.1_12183305_12183517_left_2_right_1
TTGCAGCTGAAATAGGAGCTGAAAGTCAAACATCGACGATGGTCTGTCGTATAAGAAGATACAGAGGACTCCAGACAGGATTTCGAATTAAAGAGAAAATCCTCA

5811963 5812040
Getting Gene
>Nymphalis_urticae_Exon_14_LR989982.1_5811963_5812040_left_1_right_2
GTACTTTGATTGCTATGAACCTTATGAAGATGTGAATGAAAAAAAGCTATCAATAAGAGAGGCTGGTGAAAGTAACAG
5812506 5812649
Getting Gene
>Nymphalis_urticae_Exon_15_LR989982.1_5812506_5812649_left_1_right_2
TATTTGCATGTCACTGATAGAAGAAGTACCTGATGGGTATGATGAGGGAAACTTGTATGGGGCGAGTTCGCTCATCATGTCACAATACATGCATCCCTTAGGCGACTATAAAACGTTCCGGTTGACTGAATCAGTGCTCAGCAA
5813051 5813222
Getting Gene
>Nymphalis_urticae_Exon_16_LR989982.1_5813051_5813222_left_1_right_0
ACATAACGCAGATATGGAAAAAGAATTAGTCAAAAGGCACCGTGACAACCGAATTCATAGCAAATTCGAGAGAGATACATTGTCAAGTGAAATGAAACAGAAGAAAAAAGAACATCTGTCGAGATGTAACGCCTTCATCCAACAGTCGATCCTCGAAAATGCAGAACAAAAG
5813859 5813991
Getting Gene
>Nymphalis_urticae_Exon_17_LR989982.1_5813859_5813991_left_0_right_1
CCGCAGGGTGTAAAGCGTACATCGAGGCAGAAAGAAGAAGGAAGTGCTCAAAAGCACAAATGTTCGTCTGTAAGATTAAAACGACAGAAGCAGATAGCGAGAGTTGCTAATTCACAGTCCACATCTACAATGG
5815363 5815467
Getting Gene
>Nymphalis_urticae_Exon_18_LR989982.1_5815363_58

10954606 10954707
Getting Gene
>Vanessa_atalanta_Exon_24_NC_061902.1_10954606_10954707_left_0_right_0
gTACGTGTCACTTCGgaacttatttacaaatatcaaatactCACCAAAGACGTGGATGAGGTTCTGAGTTGTGATAAGAAAAAGTTAGAAAAATTAGAACAA
10953835 10953957
Getting Gene
>Vanessa_atalanta_Exon_25_NC_061902.1_10953835_10953957_left_0_right_0
cctTCATTAGTAAACGAACAACTTGACCAATTATACTTGGATCTTCAGTTACAAGGAGTAGCTGCGAAACTAACTCTTGAGGAGGGCATCACTAGTTCCAGCAGTTCTGGTGAAGAATCGTAT
10953605 10953706
Getting Gene
>Vanessa_atalanta_Exon_26_NC_061902.1_10953605_10953706_left_0_right_0
aTTCGCCGCCGAAAGCGAGAGTATAGCAAATTAGTAATGATTTATGAAGAGGACGCCCCCCTCCCGCCTCCAGACGAGGAAGACGTTCCTTTTAGTGCGTAG
Vanessa_cardui
GCF_905220365.1_ilVanCard2.1_genomic.fna
11468772 11468848
Getting Gene
QDTENNAKVSDSAYSNSCSNSQSRR
>Vanessa_cardui_Exon_1_NC_061154.1_11468772_11468848_left_0_right_2
CAAGATACGGAAAATAATGCCAAAGTATCGGATTCAGCTTATTCTAACAGCTGTAGCAACAGCCAGTCTAGAAGAAG
11468380 11468463
Getting Gene
>Vanessa_cardui_Exon_2_NC_061154.1_11468380_11468463_left_1_right_2
ttacag

>Vanessa_tameamea_Exon_9_NW_020662887.1_22082_22310_left_2_right_2
cGGCGAATGAAATCATTGTAAATGCAAAACCATTCGTAATAAGACATTCGGCGACTGGAAGTCTAGACTATATAGATACCGAATCGGTACCTTACCTGGGCTACTTGCCTCAGGATATCGTTCAAAAGGATGCTTTGCAGCTGTATCATCCTTCAGATCTTGGATATTTGCGTCAAGTTTACGAGGCCATTGTGAAGGAAGGCGTAGTGGAGCGCTCAAAGCCTTATAG
22863 22974
Getting Gene
>Vanessa_tameamea_Exon_10_NW_020662887.1_22863_22974_left_1_right_0
gATGTTGACTCAAAATGgggattatttaaaattggaaactGAATGGTCTTCTTTTATTAACCCATGGTCCAGAAAACTTGAATTCGTAACGggcaaacattttataatagag
23107 23226
Getting Gene
>Vanessa_tameamea_Exon_11_NW_020662887.1_23107_23226_left_0_right_0
GGTCCGAGCAATCCGGACGTCTTCCAGACAGAGGAAACGGAAAGAATTGAAAAAATACCAGAGGAAGAGCAAAAGAAAGCTCAACTTTTGAGACAAAATATAACTCGAGTAATGAATGAG
24377 24532
Getting Gene
>Vanessa_tameamea_Exon_12_NW_020662887.1_24377_24532_left_0_right_0
gtTCTCACTAAGCCAGCGGAAGCTGCTAAGCAGCAAATGGGGAAAAGGTGTCAGGATTTAGCGTCGTTTATGGAGAGCCTGTTGGAAGAAACACCGAATAACAACAATGACGCGGATTTGAAACTAGAAATACAAGATCCGGACAATAATTtctat
24793 24911
Getting Gene
>Vanessa